### Generating A Drum Rhythm with NN

#### ToDo's

**17.05.2024** (Friday)
- Connnect to VS code! ✅
- Choose framework - Jax / PyTorch / Keras (not recommended for its simplicity) ✅
- Pick dataset ✅

    > Groove MIDI Dataset [Link](https://magenta.tensorflow.org/datasets/groove#download) + [Paper](https://arxiv.org/pdf/1905.06118)

- Pick model - RNN (+ MLP?) ✅
- Study the dataset and think of pre-processing it
- Study framework


**21.05.2024** (Tuesday)
- Define problem
- Define goal
- Start with setting up training model template


**23.05.2024** (Thursday)
- We code a model that acts as a generator: after running the code it should produce a simple beat pattern
- Keep things as simple as possible: use only one sounds for now
- Preprocessing and chosing the right format/encdoing is going to be the hardest part 
    - Time series: discretize, align all samples so they start on the same bar
    - Filter out different sounds (keep only one drum sounds)
    - 4/4 bar but leave space (32 time slices) for more complex pattern
    - Keep same duration for Feed-Forward Network or not for RNN
 - To keep it simple, maybe generate own data with the desired format
 - Regular meetings!!!
 - Keras is also fine
 - Understand the dataset really well before setting up the network
 - RNN not recommended if first time
 - 


 **27.05.2024** (Monday)
 - Dateset?
 - Framework division
 - Data format
 - Feed-Forward or RNN
 - Regular Meetings


#### Pre-processing 

**Resources:**

> [Short IBM Article: Data preprocessing in detail](https://developer.ibm.com/articles/data-preprocessing-in-detail/)  
> 
> [Medium Detailed Article with Example: Data pre-processing: A step-by-step guide](https://towardsdatascience.com/data-pre-processing-a-step-by-step-guide-541b083912b5)
>
> [Easy Kaggle Article: Machine Learning Episode #1: Data Pre-processing steps using Python](https://www.kaggle.com/discussions/getting-started/151612)
>
> 





#### Frameworks

**JAX**

> [Extensive Article with Example for RNN, MLP, and CNN: Getting started with JAX (MLPs, CNNs & RNNs)](https://roberttlange.com/posts/2020/03/blog-post-10/)


### Main Code

In [6]:
from pretty_midi import PrettyMIDI
import numpy as np
from pre_processing_tools import filter_to_kick_drum
from pre_processing_tools import align_all_notes_to_origin
from pre_processing_tools import is_first_note_aligned
from pre_processing_tools import make_all_notes_same_duartion
from pre_processing_tools import make_all_notes_same_volume
from pre_processing_tools import make_instrument_not_drum
from pre_processing_tools import slice_midi_to_batches
from pre_processing_tools import midi_to_bit_vector
from pre_processing_tools import change_tempo


# Initiate data structures
X_train = []
X_test = []
X_validation = []
# X_train = [0] * 897
# X_test = [0] * 129
# X_validation = [0] * 124
# y_train = np.zeros(())
# y_test = np.zeros(())

pipeline = [
    filter_to_kick_drum,
    # change_tempo,
    make_all_notes_same_duartion,
    make_all_notes_same_volume,
    make_instrument_not_drum
]

for x in X:
    x_midi = PrettyMIDI(str(x.midi_file_path))
    for process in pipeline:
        try:
            x_midi = process(x_midi)
        except Exception as er:
            break
    else:
        try:
            if not is_first_note_aligned(x_midi):
                x_midi = align_all_notes_to_origin(x_midi)    
        except:
            continue
        batches = slice_midi_to_batches(x_midi)
        # x.midi_data = x_midi
        if len(batches) == 1:
            bit_vectors = [midi_to_bit_vector(x_midi)]
        else:
            bit_vectors = []
            for batch in batches:
                # print()
                # print(batch)
                # print(*batch.instruments[0].notes, sep='\n')
                if not is_first_note_aligned(batch):
                    batch = align_all_notes_to_origin(batch)
                bit_vect = midi_to_bit_vector(batch)
                bit_vectors.append(bit_vect)
        if x.purpose == "test":
            X_test.extend(bit_vectors)
        elif x.purpose == "train":
            X_train.extend(bit_vectors)
        elif x.purpose == "validation":
            X_validation.extend(bit_vectors)
        else:
            print("Data not categorized!")
    continue

print(len(X_train))
print(len(X_test))
print(len(X_validation))

In [3]:
ratio_test = len(X_test) / (len(X_train)+len(X_test))
print(ratio_test*100)
ratio_train = len(X_train) / (len(X_train)+len(X_test))
print(ratio_train*100)



ZeroDivisionError: division by zero

In [1]:
class DataPoint:
    def __init__(self,
                 midi_data,
                 features,
                 audio_file_path,
                 midi_file_path,
                 purpose) -> None:
        self.midi_data = midi_data
        self.features = features
        self.audio_file_path = audio_file_path
        self.midi_file_path = midi_file_path
        self.purpose = purpose

feature_names = ["style", "bpm", "beat_type", "time_signature", "duration"]
feature_to_column_idx = {
    "session" : 0,
    "drummer" : 1,
    "id" : 2,
    "style" : 3,
    "bpm" : 4,
    "beat_type" : 5,
    "time_signature" : 6,
    "midi_filename" : 7,
    "audio_filename" : 8,
    "duration" : 9,
    "split" : 10
}

from pretty_midi import PrettyMIDI
from pathlib import Path


# Init for data storage
X = []

# Define filename and its path
data_filename = "info.csv"
data_folder_path = Path().cwd() / "data"
data_path = data_folder_path / data_filename

# Open file with data information
with data_path.open() as file:
    columns = file.readline()
    train_idx = 0
    test_idx = 0
    validation_idx = 0
    for i, line in enumerate(file):
        # Strip line of new line and split into list
        line = line.strip().split(",")

        bpm = line[4]
        time_signature = line[6]
        if time_signature != "4-4":
            continue

        # Attach absolute path to midi & audio files
        midi_data_path = data_folder_path / line[7]
        audio_data_path = data_folder_path / line[8]

        # Create custom data point with all info
        midi_data = PrettyMIDI(str(midi_data_path))

        features = {i: line[feature_to_column_idx[i]] for i in feature_names}
        data_point = DataPoint(
            midi_data=midi_data,
            audio_file_path=audio_data_path,
            midi_file_path=midi_data_path,
            features=features,
            purpose=line[feature_to_column_idx["split"]]
        )

        X.append(data_point)
        # print("Next sample\n")
        # if i == 10: print("Breaking"); break

In [4]:
len(X)

1138

In [ ]:
len(X_train[15])

In [ ]:
len(X_train)

In [ ]:
len(X_test)

In [ ]:
len(X_validation)

In [ ]:

# Convert to PyTorch tensors
sequences = torch.tensor(sequences, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.float32)

In [16]:
np.count_nonzero(X_train[0][36,:])

2

In [14]:
X_train[0].shape

(128, 2)

In [17]:
X_train[0][36,:]

array([254., 127.])